## Face detection using Open CV

### advanced face detection using a graphical user interface (GUI) built with Tkinter and OpenCV. The application supports both image and video face detection, allows parameter tuning, and persists user settings in a JSON fil

In [1]:
# for this we can use 

In [15]:
# import the liberaries

import cv2
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk
import numpy as np
import json
import threading
import os

In [16]:
# import the model file from opencv_zoo

# Path to YuNet ONNX model
YUNET_MODEL_PATH = "face_detection_yunet_2023mar.onnx"
YUNET_INPUT_SIZE = (320, 320)  # YuNet expects 320x320 by default

In [20]:


class FaceDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("YuNet Face Detection App")
        self.root.geometry("800x900")

        self.style = ttk.Style()
        self.style.theme_use("clam")

        # --- Define Tkinter variables BEFORE using them in widgets ---
        self.conf_threshold = tk.DoubleVar(value=0.9)
        self.nms_threshold = tk.DoubleVar(value=0.3)
        self.top_k = tk.IntVar(value=5000)
        # ------------------------------------------------------------

        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(fill=tk.BOTH, expand=True)

        self.image_tab = ttk.Frame(self.notebook)
        self.video_tab = ttk.Frame(self.notebook)
        self.settings_tab = ttk.Frame(self.notebook)

        self.notebook.add(self.image_tab, text="Image Processing")
        self.notebook.add(self.video_tab, text="Video Processing")
        self.notebook.add(self.settings_tab, text="Settings")

        self.setup_image_tab()
        self.setup_video_tab()
        self.setup_settings_tab()

        self.image = None
        self.orig_image = None
        self.video_capture = None
        self.is_processing_video = False

        # Load YuNet model
        if not os.path.isfile(YUNET_MODEL_PATH):
            messagebox.showerror("Model Error", f"YuNet model not found at {YUNET_MODEL_PATH}.\n"
                                                "Download from OpenCV Zoo and place in this directory.")
            self.root.destroy()
            return

        # Check OpenCV version
        if not hasattr(cv2, "FaceDetectorYN"):
            messagebox.showerror("OpenCV Error", "Your OpenCV version does not support FaceDetectorYN. "
                                                 "Please upgrade to OpenCV >= 4.5.4.")
            self.root.destroy()
            return

        self.face_detector = cv2.FaceDetectorYN.create(
            YUNET_MODEL_PATH, "",
            YUNET_INPUT_SIZE,
            self.conf_threshold.get(),
            self.nms_threshold.get(),
            self.top_k.get()
        )
        self.load_settings()

    def setup_image_tab(self):
        self.upload_button = ttk.Button(self.image_tab, text="Upload Image", command=self.upload_image)
        self.upload_button.pack(pady=10)

        self.detect_button = ttk.Button(self.image_tab, text="Detect Faces", command=self.detect_faces,
                                        state=tk.DISABLED)
        self.detect_button.pack(pady=10)

        self.save_button = ttk.Button(self.image_tab, text="Save Image", command=self.save_image, state=tk.DISABLED)
        self.save_button.pack(pady=10)

        self.display_label = ttk.Label(self.image_tab)
        self.display_label.pack(expand=True)

    def setup_video_tab(self):
        self.video_source = tk.StringVar(value="0")
        self.video_source_entry = ttk.Entry(self.video_tab, textvariable=self.video_source)
        self.video_source_entry.pack(pady=10)

        self.start_video_button = ttk.Button(self.video_tab, text="Start Video Processing",
                                             command=self.start_video_processing)
        self.start_video_button.pack(pady=10)

        self.stop_video_button = ttk.Button(self.video_tab, text="Stop Video Processing",
                                            command=self.stop_video_processing, state=tk.DISABLED)
        self.stop_video_button.pack(pady=10)

        self.video_label = ttk.Label(self.video_tab)
        self.video_label.pack(expand=True)

    def setup_settings_tab(self):
        ttk.Label(self.settings_tab, text="Confidence Threshold (0.0-1.0):").grid(row=0, column=0, padx=5, pady=5)
        ttk.Entry(self.settings_tab, textvariable=self.conf_threshold).grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(self.settings_tab, text="NMS Threshold (0.0-1.0):").grid(row=1, column=0, padx=5, pady=5)
        ttk.Entry(self.settings_tab, textvariable=self.nms_threshold).grid(row=1, column=1, padx=5, pady=5)

        ttk.Label(self.settings_tab, text="Top K:").grid(row=2, column=0, padx=5, pady=5)
        ttk.Entry(self.settings_tab, textvariable=self.top_k).grid(row=2, column=1, padx=5, pady=5)

        ttk.Button(self.settings_tab, text="Save Settings", command=self.save_settings).grid(row=3, column=0,
                                                                                             columnspan=2, padx=5,
                                                                                             pady=5)

    def upload_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.png *.jpeg")])
        if file_path:
            self.image = cv2.imread(file_path)
            self.orig_image = self.image.copy()
            self.show_image(self.image)
            self.detect_button.config(state=tk.NORMAL)
            self.save_button.config(state=tk.DISABLED)

    def detect_faces(self):
        if self.image is None:
            messagebox.showwarning("Warning", "Please upload an image first!")
            return

        img = self.orig_image.copy()
        height, width = img.shape[:2]
        self.face_detector.setInputSize((width, height))
        _, faces = self.face_detector.detect(img)
        if faces is not None:
            for face in faces:
                x1, y1, w, h = face[:4].astype(int)
                cv2.rectangle(img, (x1, y1), (x1 + w, y1 + h), (255, 0, 0), 2)
                # Draw landmarks if available
                for i in range(5):
                    lx, ly = int(face[4 + i * 2]), int(face[4 + i * 2 + 1])
                    cv2.circle(img, (lx, ly), 2, (0, 255, 0), -1)
        self.image = img
        self.show_image(self.image)
        self.save_button.config(state=tk.NORMAL)

    def show_image(self, cv_img):
        cv_img_rgb = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(cv_img_rgb)
        img_pil = img_pil.resize((600, 600), Image.LANCZOS)
        img_tk = ImageTk.PhotoImage(image=img_pil)
        self.display_label.imgtk = img_tk
        self.display_label.configure(image=img_tk)

    def save_image(self):
        if self.image is None:
            messagebox.showwarning("Warning", "No processed image to save!")
            return

        file_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg")])
        if file_path:
            cv2.imwrite(file_path, self.image)
            messagebox.showinfo("Success", "Image saved successfully!")

    def start_video_processing(self):
        try:
            source = int(self.video_source.get())
        except ValueError:
            source = self.video_source.get()

        self.video_capture = cv2.VideoCapture(source)
        if not self.video_capture.isOpened():
            messagebox.showerror("Error", "Could not open video source")
            return

        self.is_processing_video = True
        self.start_video_button.config(state=tk.DISABLED)
        self.stop_video_button.config(state=tk.NORMAL)

        threading.Thread(target=self.process_video, daemon=True).start()

    def stop_video_processing(self):
        self.is_processing_video = False
        if self.video_capture:
            self.video_capture.release()
        self.start_video_button.config(state=tk.NORMAL)
        self.stop_video_button.config(state=tk.DISABLED)

    def process_video(self):
        while self.is_processing_video:
            ret, frame = self.video_capture.read()
            if not ret:
                break
            height, width = frame.shape[:2]
            self.face_detector.setInputSize((width, height))
            _, faces = self.face_detector.detect(frame)
            if faces is not None:
                for face in faces:
                    x1, y1, w, h = face[:4].astype(int)
                    cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (255, 0, 0), 2)
                    for i in range(5):
                        lx, ly = int(face[4 + i * 2]), int(face[4 + i * 2 + 1])
                        cv2.circle(frame, (lx, ly), 2, (0, 255, 0), -1)
            self.show_video_frame(frame)
        self.video_label.configure(image=None)

    def show_video_frame(self, frame):
        cv_img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(cv_img_rgb)
        img_pil = img_pil.resize((600, 600), Image.LANCZOS)
        img_tk = ImageTk.PhotoImage(image=img_pil)
        self.video_label.imgtk = img_tk
        self.video_label.configure(image=img_tk)

    def save_settings(self):
        settings = {
            "conf_threshold": self.conf_threshold.get(),
            "nms_threshold": self.nms_threshold.get(),
            "top_k": self.top_k.get()
        }
        with open("face_detection_settings.json", "w") as f:
            json.dump(settings, f)
        messagebox.showinfo("Success", "Settings saved successfully!")
        # Update detector with new settings
        self.face_detector = cv2.FaceDetectorYN.create(
            YUNET_MODEL_PATH, "",
            YUNET_INPUT_SIZE,
            self.conf_threshold.get(),
            self.nms_threshold.get(),
            self.top_k.get()
        )

    def load_settings(self):
        try:
            with open("face_detection_settings.json", "r") as f:
                settings = json.load(f)
            self.conf_threshold.set(settings.get("conf_threshold", 0.9))
            self.nms_threshold.set(settings.get("nms_threshold", 0.3))
            self.top_k.set(settings.get("top_k", 5000))
        except FileNotFoundError:
            pass

def main():
    root = tk.Tk()
    app = FaceDetectionApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
